In [1]:
import tensorflow as tf

import CNN # External class generated for CNN

import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Disable eager execution
tf.compat.v1.disable_eager_execution()


epochs=100
batch_size=250


eps_FSGM=0.10

# Select the attacker
#White-box: "FGSM", "BIM", "PGD", 
#Black-box: "SA","ZA"
attack_type = "FGSM"
dataset="MNIST"   # MNIST, CIRFAR10

#path='results/MNIST_BA.txt' #this for MNIST images
path='C:/Users/02729/OneDrive - Universiti Teknikal Malaysia Melaka/Desktop/IEEE Access paper/MNIST/results1/MNIST/MNIST_FGSM.txt' 

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\art\estimators\certification\__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:
# Load and preprocess the MNIST dataset
from art.utils import load_mnist
(train_images, train_labels), (test_images, test_labels), min_pixel_value, max_pixel_value = load_mnist()
train_images, test_images = train_images / max_pixel_value, test_images / max_pixel_value


'''# Load CIFAR-10 dataset
from tensorflow.keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Normalize pixel values to [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

min_pixel_value = 0.0
max_pixel_value = 1.0'''



classifier, t_consumed = CNN.train(train_images, train_labels, epochs, batch_size,
                                   min_pixel_value, max_pixel_value)

Train on 60000 samples
60000/60000 [==============================] - 18s 295us/sample - loss: 0.3826 - accuracy: 0.8769
Done...Time consumed in training: 18.43698024749756


In [3]:
test_labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [4]:
#Evaluate the model
acc, prc, rec, f1 = CNN.test(classifier, test_images, test_labels)

#write the results
import Writer
results ="===========\n" +  dataset + "-Regular:\n acc, prc, rec, f1, epochs, batch size, time: acc, prc, rec, f1\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(epochs)+ "," + str(batch_size) + "," + str(int(t_consumed)) +"\n" 
Writer.write_results(results, path)


C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


acc, prc, rec, f1: 97.72 97.75 97.72 97.72


In [5]:
# Extract the features for the regular images
from tensorflow.keras import models

model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_regular = model_output.predict(test_images)
print(Features_regular.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [6]:
# Generate adversarial examples using the FastGradientMethod attack
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, ProjectedGradientDescent #white-box
from art.attacks.evasion import ZooAttack, SquareAttack, BoundaryAttack #black-box


# Initialize the attack
attack = None

# Select the attack
if attack_type == "FGSM":
    attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
if attack_type == "BIM":
    attack = BasicIterativeMethod(estimator=classifier, eps=eps_FSGM)
elif attack_type == "PGD":
    attack = ProjectedGradientDescent(estimator=classifier, eps=eps_FSGM)
elif attack_type == "SA":
    attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
elif attack_type == "ZA":
    attack = ZooAttack(classifier=classifier, max_iter=2, learning_rate=eps_FSGM)



adv_images = attack.generate(test_images)


In [7]:
# Evaluate the model on the adversarial test set images
acc, prc, rec, f1 = CNN.test(classifier, adv_images, test_labels)

#write the results
results =dataset + "-Adversarial:\n acc, prc, rec, f1, epochs, attack_type, eps_FSGM\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + attack_type + "," + str(eps_FSGM) +"\n" 
Writer.write_results(results, path)

acc, prc, rec, f1: 84.02 84.6 84.02 84.0


In [8]:
# Extract the features for the adversarial images
model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_adversarial = model_output.predict(adv_images)
print(Features_adversarial.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [9]:
# merge regular and feature test images
from sklearn.model_selection import train_test_split

#merge data features of regular and adversarial images
reg_adv_images = np.concatenate((Features_regular, Features_adversarial), axis=0)

# generate labels for regular and adversarial, and merge them

label_reg = np.zeros((10000, 1))
label_adv = np.ones((10000, 1))
reg_adv_labels=np.concatenate((label_reg, label_adv), axis=0)

print("data details: ", reg_adv_images.shape, " , labels: ", reg_adv_labels.shape)

X_train, X_test, y_train, y_test = train_test_split(reg_adv_images, reg_adv_labels, test_size=0.20, random_state=42)

#fractin to use a part of the merged regular and adversarial images
fraction=1
#reduce the length because the memory is out
X_train=X_train[:int(X_train.shape[0]/fraction)]
X_test=X_test[:(int(X_test.shape[0]/fraction))]
y_train=y_train[:(int(y_train.shape[0]/fraction))]
y_test=y_test[:(int(y_test.shape[0]/fraction))]
X_train, X_test, y_train, y_test
print("data after split : X_train, X_test, y_train, y_test\n", X_train.shape, X_test.shape, y_train.shape, y_test.shape)


data details:  (20000, 32)  , labels:  (20000, 1)
data after split : X_train, X_test, y_train, y_test
 (16000, 32) (4000, 32) (16000, 1) (4000, 1)


In [10]:
'''import ML_Classifiers as MLC

RF_Pred= MLC.Random_Forest(X_train, X_test, y_train, y_test)
DT_Pred= MLC.DT(X_train, X_test, y_train, y_test)
KNN_Pred= MLC.KNN(X_train, X_test, y_train, y_test)
XGB_Pred= MLC.XGB(X_train, X_test, y_train, y_test)
NN_Pred= MLC.NN(X_train, X_test, y_train, y_test)
NN_Pred= MLC.GBM(X_train, X_test, y_train, y_test)'''


'import ML_Classifiers as MLC\n\nRF_Pred= MLC.Random_Forest(X_train, X_test, y_train, y_test)\nDT_Pred= MLC.DT(X_train, X_test, y_train, y_test)\nKNN_Pred= MLC.KNN(X_train, X_test, y_train, y_test)\nXGB_Pred= MLC.XGB(X_train, X_test, y_train, y_test)\nNN_Pred= MLC.NN(X_train, X_test, y_train, y_test)\nNN_Pred= MLC.GBM(X_train, X_test, y_train, y_test)'

In [11]:
# integrate the adversarial detector model to CNN to exclude adversarial images or outlier images.

size_advs=10000
test_reg_adv_images=np.concatenate((test_images,adv_images[:size_advs]), axis=0)
test_reg_adv_labels=np.concatenate((test_labels,test_labels[:size_advs]), axis=0)
print(test_reg_adv_images.shape, test_reg_adv_labels.shape)

# Shuffle the data
indices = np.arange(len(test_reg_adv_images))
np.random.shuffle(indices)
test_reg_adv_images= test_reg_adv_images[indices]
test_reg_adv_labels = test_reg_adv_labels[indices]

# Evaluate the model after integrate regular and adversarial images
acc, prc, rec, f1 = CNN.test(classifier, test_reg_adv_images, test_reg_adv_labels)
results =dataset + "-Regular+Adversarial:\n acc, prc, rec, f1\n"
results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) +"\n" 
Writer.write_results(results, path)


(20000, 28, 28, 1) (20000, 10)
acc, prc, rec, f1: 90.87 91.11 90.87 90.87


In [12]:
# Extract features 
model_output = models.Model(inputs=classifier._model.input, outputs=classifier._model.get_layer('dense_2').output)
Features_adv_reg = model_output.predict(test_reg_adv_images)
print(Features_adversarial.shape)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(10000, 32)


In [13]:
#to ensure that the size of regular and adversarial image size are the same to do fair evaluation
def make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels):
    diff=int(np.absolute(clean_images.shape[0] - attacked_images.shape[0])/2)
    size=int((clean_images.shape[0]+attacked_images.shape[0])/2)
    temp_clean_images=np.ones((size,28,28,1))
    temp_attacked_images=np.ones((size,28,28,1))

    temp_clean_labels=np.ones((size,10))
    temp_attacked_labels=np.ones((size,10))
    
    if(clean_images.shape[0]>attacked_images.shape[0]):
        temp_clean_images=clean_images[:size]
        temp_clean_labels=clean_labels[:size]
        
        temp_attacked_images=np.concatenate((attacked_images,clean_images[-diff:]), axis=0)
        temp_attacked_labels=np.concatenate((temp_attacked_labels,clean_labels[-diff:]), axis=0)
        
    elif(clean_images.shape[0]<attacked_images.shape[0]):
        temp_attacked_images=attacked_images[:size]
        temp_attacked_labels=attacked_labels[:size]

        temp_clean_images=np.concatenate((clean_images,attacked_images[-diff:]), axis=0)
        temp_clean_labels=np.concatenate((clean_labels,attacked_labels[-diff:]), axis=0)

    clean_images=temp_clean_images
    clean_labels=temp_clean_labels
    
    attacked_images=temp_attacked_images
    attacked_labels=temp_attacked_labels
    
    #print("clean_images.shape, attacked_images.shape", clean_images.shape, attacked_images.shape)
    #print("clean_labels.shape, attacked_labels.shape", clean_labels.shape, attacked_labels.shape)
    
    return clean_images,clean_labels, attacked_images, attacked_labels

#Exclude adversarial images from testing and use only the regular images

def retrurn_clean_images(adv_images_pred_binary):
    
    # Count the number of times 1 appears in the array
    number_of_1s = np.count_nonzero(adv_images_pred_binary == 1)
    # Count the number of times 0 appears in the array
    number_of_0s = np.count_nonzero(adv_images_pred_binary == 0)
    print("The number of attacked immages 1 is:", number_of_1s)
    print("The number of clean images 0 is:", number_of_0s)

    #Exclude adversarial images from testing and use only the regular images
    # Create a boolean mask use only the clean images
    mask_clean = np.where(adv_images_pred_binary == 0)[0]
    mask_adv = np.where(adv_images_pred_binary == 1)[0]

    # Exclude adversarial images
    #clean images
    clean_images = test_reg_adv_images[mask_clean]
    clean_labels = test_reg_adv_labels[mask_clean]
    #Adversarial images,attacked images
    attacked_images = test_reg_adv_images[mask_adv]
    attacked_labels = test_reg_adv_labels[mask_adv]
    
    clean_images,clean_labels, attacked_images, attacked_labels = make_size_fit(clean_images,clean_labels, attacked_images, attacked_labels)
    
    print("clean_images:", clean_images.shape)
    print("clean_labels:", clean_labels.shape)
    
    return clean_images, clean_labels


def eval_model_after_exclude_adv_image(model_name, new_test_clean_images, new_test_clean_labels, t_consumed):# Evaluate the model after excluding the adversarial images
    acc, prc, rec, f1 = CNN.test(classifier, new_test_clean_images, new_test_clean_labels)
    results =model_name + ":\n acc, prc, rec, f1, time_detection\n"
    results +=str(acc) + "," + str(prc) + "," + str(rec)+ "," + str(f1) + "," + str(int(t_consumed)) + "\n" 
    Writer.write_results(results, path)

In [14]:
#Evaluate the mosdel after inegrate NN as outlier detection to exclude Adversarial images
import ML_Classifiers as MLC
t_str=time.time()

adv_images_pred_binary= MLC.NN(X_train, X_test, y_train, y_test,eps_FSGM, attack_type, Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+NN_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

Train on 16000 samples
Epoch 1/50
16000/16000 [==============================] - 1s 34us/sample - loss: 0.6419 - accuracy: 0.6432
Epoch 2/50
16000/16000 [==============================] - 1s 33us/sample - loss: 0.5318 - accuracy: 0.7378
Epoch 3/50
16000/16000 [==============================] - 0s 26us/sample - loss: 0.4847 - accuracy: 0.7726
Epoch 4/50
16000/16000 [==============================] - 0s 29us/sample - loss: 0.4578 - accuracy: 0.7847
Epoch 5/50
16000/16000 [==============================] - 0s 27us/sample - loss: 0.4367 - accuracy: 0.7951
Epoch 6/50
16000/16000 [==============================] - 1s 33us/sample - loss: 0.4204 - accuracy: 0.8054
Epoch 7/50
16000/16000 [==============================] - 0s 25us/sample - loss: 0.4045 - accuracy: 0.8195
Epoch 8/50
16000/16000 [==============================] - 0s 26us/sample - loss: 0.3960 - accuracy: 0.8222
Epoch 9/50
16000/16000 [==============================] - 0s 27us/sample - loss: 0.3842 - accuracy: 0.8273
Epoch 10/50
16

C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:251: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 10426
The number of clean images 0 is: 9574
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)
acc, prc, rec, f1: 97.51 97.52 97.51 97.51


In [15]:
##Evaluate the mosdel after inegrate RF as outlier detection to exclude Adversarial images
import ML_Classifiers as MLC
t_str=time.time()

adv_images_pred_binary= MLC.Random_Forest(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+RF_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf_rf.fit(X_train, y_train)


RF detection: acc, prc, rec, f1: 84.52 84.98 84.52 84.48


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:38: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 10225
The number of clean images 0 is: 9775
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)
acc, prc, rec, f1: 97.57 97.6 97.57 97.57


In [16]:
##Evaluate the mosdel after inegrate DT as outlier detection to exclude Adversarial images
t_str=time.time()
adv_images_pred_binary= MLC.DT(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+DT_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

DT detection: acc, prc, rec, f1: 74.3 74.3 74.3 74.3
The number of attacked immages 1 is: 9986
The number of clean images 0 is: 10014
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:113: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 97.08 97.13 97.08 97.08


In [17]:
##Evaluate the mosdel after inegrate KNN as outlier detection to exclude Adversarial images
t_str=time.time()
adv_images_pred_binary= MLC.KNN(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+KNN_outlier_detection", new_test_clean_images, new_test_clean_labels,t_consumed)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNN detection: acc, prc, rec, f1: 79.2 79.85 79.2 79.11


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


The number of attacked immages 1 is: 11252
The number of clean images 0 is: 8748
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)
acc, prc, rec, f1: 96.85 96.9 96.85 96.85


In [18]:
##Evaluate the mosdel after inegrate GBM as outlier detection to exclude Adversarial images
t_str=time.time()

adv_images_pred_binary= MLC.GBM(X_train, X_test, y_train, y_test,Features_adv_reg)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+GBM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

C:\Users\02729\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GBM detection: acc, prc, rec, f1: 78.08 78.69 78.08 77.98
The number of attacked immages 1 is: 11435
The number of clean images 0 is: 8565
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 96.1 96.15 96.1 96.1


In [20]:
##Evaluate the mosdel after inegrate XGB as outlier detection to exclude Adversarial images
t_str=time.time()

adv_images_pred_binary= MLC.XGB(X_train, X_test, y_train, y_test,Features_adv_reg)

new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+XGB_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)

XGB detection: acc, prc, rec, f1: 69.05 69.91 69.05 68.76
The number of attacked immages 1 is: 11986
The number of clean images 0 is: 8014
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\ML_Classifiers.py:160: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


acc, prc, rec, f1: 94.64 94.75 94.64 94.64


In [21]:
##Evaluate the mosdel after inegrate FAM as outlier detection to exclude Adversarial images
import FAM 

t_str=time.time()
ArtMap=FAM.train(X_train[:5000], y_train[:5000])

adv_images_pred_binary = FAM.test(X_test[:1000], y_test[:1000], ArtMap, Features_adv_reg)

adv_images_pred_binary=np.array(adv_images_pred_binary)
new_test_clean_images, new_test_clean_labels = retrurn_clean_images(adv_images_pred_binary)

t_consumed=time.time()-t_str
eval_model_after_exclude_adv_image("CNN+FAM_outlier_detection", new_test_clean_images, new_test_clean_labels, t_consumed)


378  values were replaced from  1000
FAM detection: acc, prc, rec, f1: 60.7 58.14 70.14 63.58


C:\Users\02729\OneDrive - Universiti Teknikal Malaysia Melaka\Desktop\IEEE Access paper\MNIST\FAM.py:71: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if(adv_reg_images!=""):


{'index': 1, 'ArtB': [1.0, 0.0], 'id': '1000'}
The number of attacked immages 1 is: 11542
The number of clean images 0 is: 8458
clean_images: (10000, 28, 28, 1)
clean_labels: (10000, 10)
acc, prc, rec, f1: 92.26 92.39 92.26 92.25


In [ ]:
'''
from art.attacks.evasion import ZooAttack
attack = ZooAttack(classifier=classifier)



#attack = FastGradientMethod(estimator=classifier, eps=eps_FSGM)
adv_images = attack.generate(test_images)

attack = SquareAttack(estimator=classifier,eps=eps_FSGM)
adv_images = attack.generate(test_images)

'''